In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [2]:
!pip install -q datasets evaluate wandb

In [4]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, classification_report
import evaluate
import wandb
import os
import random
import numpy as np
import torch

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
SEED = 42
NUM_PROC=5
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [7]:
#EXPERIMENT_NAME = "MBE"
CACHE_DIR = "/content/drive/MyDrive/.cache"
LEARNING_RATE=1e-6
TRAINING_BATCH=64
WARMUP=0.1
DECAY=0.01
NUM_TRAIN_EPOCHS=6
MODEL_SAVE_PATH = f"/content/drive/MyDrive/mbert_{LEARNING_RATE}_{TRAINING_BATCH}_{WARMUP}_{DECAY}"

In [8]:
EXPERIMENT_NAME="MBERT"
os.environ["WANDB_PROJECT"]=f"{EXPERIMENT_NAME}_train"
wandb.login()
wandb.init(
    entity="mbzuai-ugrip",
    name=f"{EXPERIMENT_NAME}_{LEARNING_RATE}_{TRAINING_BATCH}_{WARMUP}_{DECAY}",
    project=f"{EXPERIMENT_NAME}_train"
)

wandb: Currently logged in as: soph20 (mbzuai-ugrip). Use `wandb login --relogin` to force relogin


In [9]:
TRANSFORMER="google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
tolerance = 20
data = load_dataset('ashabrawy/STTS', cache_dir=CACHE_DIR)
train = data['train'].filter(lambda example: example["is_true"] is not None).filter(lambda example: len(tokenizer(example['statement'])['input_ids']) < 514+tolerance)

In [11]:
train = train.train_test_split(test_size=50000)['test']

In [12]:
train_statements, val_statements, train_labels, val_labels = train_test_split(train['statement'], train['is_true'], test_size=.1)

In [13]:
class StatementDataset(torch.utils.data.Dataset):
    def __init__(self, statements, labels):
        self.statements = statements
        self.labels = labels

    def __getitem__(self, idx):
        encodings = tokenizer(self.statements[idx], truncation=True, padding=True)
        item = {key: torch.tensor(val) for key, val in encodings.items()}
        item['labels'] = int(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = StatementDataset(train_statements, train_labels)
val_dataset = StatementDataset(val_statements, val_labels)

In [15]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries, which is in ids into text
    _, predictions = torch.max(torch.tensor(predictions), dim=1)


    return clf_metrics.compute(predictions=predictions, references=labels)

In [17]:
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [18]:
training_args = TrainingArguments(
    output_dir=f'./{EXPERIMENT_NAME}-outputs',          # output directory
    num_train_epochs=NUM_TRAIN_EPOCHS,              # total number of training epochs (num train epochs: 6)
    per_device_train_batch_size=TRAINING_BATCH,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_ratio=WARMUP,                # number of warmup steps for learning rate scheduler
    learning_rate=LEARNING_RATE,
    weight_decay=DECAY,               # strength of weight decay
    logging_dir=f'./{EXPERIMENT_NAME}-logs',            # directory for storing logs
    logging_steps=1000,
    save_steps=1000,
    evaluation_strategy='steps',
    save_total_limit=2,
    load_best_model_at_end= True,
    metric_for_best_model='f1',
    report_to="wandb",
)

model = AutoModelForSequenceClassification.from_pretrained(TRANSFORMER)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    compute_metrics=compute_metrics,
    eval_dataset=val_dataset,            # evaluation dataset
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


OutOfMemoryError: CUDA out of memory. Tried to allocate 372.00 MiB. GPU 

In [ ]:
trainer.save_model(MODEL_SAVE_PATH)

In [ ]:
trainer.push_to_hub("sophiayk20/mbert-english-model")